# Dataquest Guided Project: Profitable App Profiles for the App Store and Google Play Markets

The goal of this project is to analyize Android and IOS mobile apps. The company we represent builds **free** apps directed towards an **English-speaking** audience, with revenue coming from in-app advertising.

The goal of the project is to learn which types of applications are most likely to attract more users.

## About the Data

* [A data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

### Import the Data

In [1]:
import tarfile

# Extracts the .tar file containing both datasets
app_data_tgz = tarfile.open('Guided Project_ Profitable App Profiles for the App Store and Google Play Markets.tar')
app_data_tgz.extractall()
app_data_tgz.close()

Open the two data sets and save them to list variables:

In [2]:
def open_csv_dataset(file, header=True):
    """Converts a CSV file into a Python list for analysis.
     Parameters:
        - file: file name
        - header=True: True if first row of data represents categories.
        
    Returns:
        - header, list (if header=True)
        - list (if header=False)
        """
    from csv import reader
    open_file = open(file, encoding='utf-8')
    read_file = reader(open_file)
    name = list(read_file)
    open_file.close()
    
    if header:
        col_names = name[0]
        return col_names, name[1:]
    else:
        return name

In [3]:
IOS_COLS, IOS_DATA = open_csv_dataset('Applestore.csv')
ANDROID_COLS, ANDROID_DATA = open_csv_dataset('googleplaystore.csv')

In [4]:
help(open_csv_dataset)

Help on function open_csv_dataset in module __main__:

open_csv_dataset(file, header=True)
    Converts a CSV file into a Python list for analysis.
     Parameters:
        - file: file name
        - header=True: True if first row of data represents categories.
        
    Returns:
        - header, list (if header=True)
        - list (if header=False)



In [5]:
def explore_data(dataset, start, end, rows_and_columns=False):
    """Function to explore a dataset based on the specified indices
    
    Parameters:
        - dataset: variable representing the dataset
        - start: start index within dataset
        - end: stop index within dataset
        - rows_and_columns: prints the number of rows and columns, if True
        
    Returns:
        None
        """
    
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # Adds a new line after each row
        
    if rows_and_columns:
        print(f'Number of rows: {len(dataset)}')
        print(f'Number of columns: {len(dataset[0])}')

### Explore Datasets

In [6]:
explore_data(IOS_DATA, 3, 5, rows_and_columns=True)

['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16


In [7]:
explore_data(ANDROID_DATA, 3, 5, rows_and_columns=True)

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 10841
Number of columns: 13


### Explore Column Names

In [8]:
# IOS Columns:
for col in IOS_COLS:
    print(col)

id
track_name
size_bytes
currency
price
rating_count_tot
rating_count_ver
user_rating
user_rating_ver
ver
cont_rating
prime_genre
sup_devices.num
ipadSc_urls.num
lang.num
vpp_lic


Of these IOS columns, the following might be useful for our purpose:
* price
* user_rating
* cont_rating (Maybe)
* prime genre
* sup_devices.num (Maybe)
* lang.num (Maybe)

[Link to data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [9]:
# Android Columns:
for col in ANDROID_COLS:
    print(col)

App
Category
Rating
Reviews
Size
Installs
Type
Price
Content Rating
Genres
Last Updated
Current Ver
Android Ver


Of these Android columns, the following might be useful:
* Category
* Rating
* Reviews
* Size
* Installs
* Type
* Content Rating
* Genres

Price should be superfluous, as we will hone in on type=Free.

### Data Cleaning

Data cleaning is done before the analysis; it includes removing or correcting wrong data, removing duplicate data, and modifying the data to fit the purpose of our analysis.

**To-Do:**
1. Identify the issue with a well-known error in one of the Android rows.
    * [Discussion Source](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015)
    
Data:
* IOS_COLS, IOS_DATA 
* ANDROID_COLS, ANDROID_DATA

In [10]:
# Labels:
print(ANDROID_COLS)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [11]:
def error_checker(dataset, dataset_header=None):
    """Checks for length inconsistencies within the dataset.
    
    Parameters:
        - dataset_header: Header (labels) of the data, if available
        - dataset: dataset (list)
        
    Returns:
        - error_list: nested list of errors. Includes:
            * error[0]: Data values (to help with inspection)
            * error[1]: Index of the error within the dataset
    """
    error_list = []
    if not dataset_header:
        dataset_header = dataset[0]
        
    for row in dataset:
        if len(row) != len(dataset_header):
            error = []
            idx = dataset.index(row)
            error.append(row)
            error.append(idx)
            error_list.append(error)
            print(f'Data error found at index: {idx}')
    num_errors = len(error_list)
    print(f'{num_errors} length inconsistencies in this dataset')
    return error_list

In [12]:
# for row in ANDROID_DATA:
#     if len(row) != len(ANDROID_COLS):
#         print(row)
#         print(f'The index is: {ANDROID_DATA.index(row)}')

error_checker(ANDROID_DATA, ANDROID_COLS)

Data error found at index: 10472
1 length inconsistencies in this dataset


[[['Life Made WI-Fi Touchscreen Photo Frame',
   '1.9',
   '19',
   '3.0M',
   '1,000+',
   'Free',
   '0',
   'Everyone',
   '',
   'February 11, 2018',
   '1.0.19',
   '4.0 and up'],
  10472]]

#### Findings:
* "Genres" is missing from index 10472 in the ANDROID_Data. I have decided to delete this row from the dataset, rather than augment it.

In [13]:
ANDROID_DATA[10472]

['Life Made WI-Fi Touchscreen Photo Frame',
 '1.9',
 '19',
 '3.0M',
 '1,000+',
 'Free',
 '0',
 'Everyone',
 '',
 'February 11, 2018',
 '1.0.19',
 '4.0 and up']

In [14]:
del(ANDROID_DATA[10472])

In [15]:
android_errors = error_checker(ANDROID_DATA, ANDROID_COLS)

0 length inconsistencies in this dataset


In [16]:
ios_errors = error_checker(IOS_DATA, IOS_COLS)

0 length inconsistencies in this dataset


### Check for Redundant Apps

In [17]:
def find_duplicates(dataset, attr_idx):
    """Checks a dataset list for multiple occurrences of a row
    
    Parameters:
        - dataset: (list) Dataset, excluding headers
        - attr_idx (int) Index of the data attribute
        
    Returns:
        - num_duplicates (int): Number of duplicate entries within the dataset.
        - duplicate_apps (list): List of duplicate apps
    
    """
    duplicate_apps = []
    unique_apps = []
    
    for row in dataset:
        name = row[0]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    
    num_duplicates = len(duplicate_apps)
    print(f'{num_duplicates} duplicates found')
    
    return num_duplicates, duplicate_apps

In [18]:
android_num_duplicates, android_duplicates = find_duplicates(ANDROID_DATA, 0)

1181 duplicates found


In [19]:
ios_num_duplicates, ios_duplicates = find_duplicates(IOS_DATA, 0)

0 duplicates found


As shown above, the `ANDROID_DATA` has 1181 duplicates, while the `IOS_DATA` has none. 

Within these duplicates, all fields are the same except for the number of reviews (index 3). My strategy will be to keep the entry with the highest number of reviews, discarding the others. This should be reasonable, as the remaining entry will be the most frequently-updated representation of the app.

In [20]:
print(ios_num_duplicates)

0


### Remove Duplicate Android Entries

Verify the target length of a unique dictionary for ANDROID_DATA:

In [21]:
print('Number of apps:', len(ANDROID_DATA) - 1181)

Number of apps: 9659


**Target length is 9659**

In [22]:
# Explore index/columns within ANDROID_DATA:
print(list(enumerate(ANDROID_COLS)))

[(0, 'App'), (1, 'Category'), (2, 'Rating'), (3, 'Reviews'), (4, 'Size'), (5, 'Installs'), (6, 'Type'), (7, 'Price'), (8, 'Content Rating'), (9, 'Genres'), (10, 'Last Updated'), (11, 'Current Ver'), (12, 'Android Ver')]


I will now: 
* Create a blank dictionary, `reviews_max`
* Iterate through ANDROID_DATA, assigning variables for `name` and `n_reviews`
* If the app is already included in `reviews_max` and the number of reviews is greater than the existing value, the `n_reviews` value is increased accordingly.
* If the app is **not** already in `reviews_max`, it is added with the corresponding `n_reviews` value.
*Note: If the app is already included in the dictionary but it has a lower n_reviews value, it is ignored. The goal is to populate the dictionary with each app having the most reviews.

In [23]:
reviews_max = dict()
for app in ANDROID_DATA:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and n_reviews > reviews_max[name]:
        reviews_max[name] = n_reviews
    if name not in reviews_max:   # "else" statement wouldn't account for existing entries with HIGHER reviews
        reviews_max[name] = n_reviews

In [24]:
len(reviews_max)

9659

### Remove Non-English Apps

Our goal is to only analyize free, *english* applications.

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

#### ord(str) and chr(int)

In [25]:
ord('a')

97

In [26]:
chr(497)

'Ǳ'

#### Create a function to detect non-english characters

In [27]:
def is_english(string):
    for c in string:
        if ord(c) > 127:
            return False
    return True

Test function:

In [28]:
test_1 = '爱奇艺PPS -《欢乐颂2》电视剧热播'
test_2 = 'Docs To Go™ Free Office Suite'
test_3 = 'Instachat 😜'
test_4 = 'Instagram'

In [29]:
test_list = [test_1, test_2, test_3, test_4]

for test in test_list:
    print(is_english(test))

False
False
False
True


To minimize the impact of data loss, we'll only remove an app if its name has more than three characters outside of ASCII range 0-127

#### Redefine `is_english` to Filter Only Strings with More Than Three Abnormal ASCII Characters

In [30]:
def is_english(string):
    num_abnormal_ascii = 0
    for c in string:
        if ord(c) > 127:
            num_abnormal_ascii += 1
    return False if num_abnormal_ascii > 3 else True

In [31]:
is_english('Docs To Go™ Free Office Suite')

True

In [32]:
is_english('Instachat 😜😜')

True

In [33]:
is_english('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

As a reminder, the current datasets that we are working with are as follows:
* `ANDROID_DATA` and `ANDROID_COLS`
* `IOS_DATA` and `IOS_COLS`

Now, let's use our new `is_english` function to filter non-english apps.